In [1]:
import pymongo
import re
import pprint
import json
import jieba
from collections import Counter
import time
import math
import logging
from gensim.models import word2vec
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

D:\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


# 將資料由pymongo讀取近來

In [2]:
# #登入pymongo
# clinet=pymongo.MongoClient("mongodb://10.120.27.23:27017/")
# #確認裡面有哪幾個database
# clinet.database_names()
# #選擇rawData這個db
# db=clinet['rawData']
# #確認裡面有哪幾個collection
# db.collection_names()
# #將愛評網讀入
# ipeenjson=list(db.Tainan_final.find({}))

 # 或直接讀取json檔

In [6]:
with open(r'./data/final_tainan.json') as f:
    ipeenjson=json.load(f)

In [7]:
ipeenjson[0]

{'_id': ObjectId('59afa8e6f3a322258443a5e3'),
 'address': '台南市安南區安北路433號',
 'average_consumption': '42',
 'business_hours': '09:00~23:00',
 'comment': [{'click': 1004,
   'content': '簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji

# 將結巴字典加入自己的許多詞

In [8]:

#結巴 匯入自己的詞典
jieba.set_dictionary('./dict/jieba_dict.txt.big')

#停止詞字典stopwordset
stopwordset = set()
with open('./dict/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))
        
#店家名稱字典dienlist
dienlist = [dien["name"] for dien in ipeenjson]

#店家種類字典stylelist
stylelist = [dien["style"] for dien in ipeenjson]

#店家種類字典introductionlist
introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
coordinatelist = [dien['coordinate'] for dien in ipeenjson]

#自製餐廳相關字字典pinlist
with open("./dict/MyDict.csv") as f:
    mydict=f.read()
#linewords pindict每個字的評分dict
linewords=[{myword:float(line.split(",")[0]) for myword in line.split(",")[1:]}\
           for line in mydict.split("\n") if line.split(",")[0]!=""]
pindict={}
for word in linewords:
    pindict.update(word)
pinlist=list(pindict.keys())

#將以上字典都加入斷詞行列
for i in dienlist:
    jieba.add_word(i)
for i in stylelist:
    jieba.add_word(i)
for i in pinlist:
    if i !="":
        jieba.add_word(i)
        
#店家評論總數Ncommentlist
Ncommentlist = [len(i["comment"]) for i in ipeenjson]


    

Building prefix dict from D:\WordLibrary\JiebaUse\jieba_dict.txt.big ...
Loading model from cache C:\Users\Java\AppData\Local\Temp\jieba.uf3aabe1eaf7c86dda8313189b0c5c683.cache
Loading model cost 1.057 seconds.
Prefix dict has been built succesfully.


In [9]:
#各店家評論diencommentlist
diencommentlist=[[j["content"] for j in i["comment"]] for i in ipeenjson]
diencommentlist[0]

['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [10]:
#各店家回覆dienmessagelist
dienmessagelist=[[''.join(j["message"]) for j in i["comment"] if j["message"]!=None and j["message"]!='null'] for i in ipeenjson]
dienmessagelist[0]

['感謝分享，也幫我推一個，謝謝http://www.ipeen.com.tw/comment/943950',
 '到台南好像不能錯過安平的豆花耶給你非常實用也希望你能給我實用喔【電影類別 / 動作】玩命關頭7-復仇之路 回歸到起點推~~~同是台南人XD這家真的好吃http://www.ipeen.com.tw/comment/838440麻煩支持 一下喔:)',
 '鍋燒意麵這款中山店沒有賣!!!!',
 '竹炭豆畫...黑色的嘛==?感覺鮮奶的比較好吃去安平，必吃美食~~讚！',
 '好像我去過這家！很面熟，但是不太確定。',
 '回推囉，豆花看起來不錯呢！',
 '感覺你們玩超多天的耶XD呵呵 我跟阿左去三天兩夜都感覺沒有你們玩得多呢我喜歡粉圓豆花。這一家商妮去台南玩的時候也有去吃，人好多~',
 '豆花看起來好好吃><紅豆在上面顏色就看起來很漂亮最近的美食分享，喜歡的話幫忙給個讚吧非常謝謝您GODIVA比利時巧克力●奢華享受http://www.ipeen.com.tw/comment/611570IOU Cafe●甜點美饌 http://www.ipeen.com.tw/comment/611714安平豆花我也超愛~下次再來台南~安平豆花地鍋燒意麵和對面的福記水煎包也都蠻好吃的~~很特別的豆花.看起來很嫩那天下去看到好多人，就沒有去排了!!!',
 '每次去安平 我都必吃那家豆花耶~~~超愛!!',
 '我喜歡粉圓加檸檬，酸酸甜甜又清涼上次去台南很猶豫要吃這間還是黑豆花，結果選了黑豆花太太太銷魂了謝謝分享',
 '超愛珍珠豆花的啦~~~~~~檸檬也蠻好吃的~隔壁鍋燒意麵也好吃',
 '好久沒吃了! 上次很像點珍珠檸檬豆花~好吃也喜歡酸甜感我上次也是點檸檬的耶!  紅豆看起來挺不賴的這不是排隊排慘的那家嗎XD豆花好綿啊~看起來就很好吃可惜我上回去~找不到這家，最後隨便路邊找了一間進去吃，結果難吃到爆~~~我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒出名時，我就常去那吃，當時是家小小的店，店裡桌以才3~4張而已呢檸檬的超好吃!!!他們家的鍋燒意面也很不錯~下次可以試試~!\t※ 引述 ✿花露露✿ 之言：\t檸檬的超好吃!!!\t他們家的鍋燒意面也很不錯~下次可以試試~!蝦密??還有賣鍋燒意麵阿?真的假的去很多次了耶~都沒發現....XD\t※

In [11]:
#回覆數量
Nmessagelist=[len(dienmessage) for dienmessage in dienmessagelist]

In [12]:
#全部回覆+全部評論
dienallpinlist=[diencomment+dienmessage for diencomment,dienmessage in zip(diencommentlist,dienmessagelist)]
dienallpinlist[0]


['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [13]:
# for megs in dienmessagelist:
#     for meg in megs:
#         if meg!=None and meg!='null'and meg!='':
#             print(" ".join(meg))

#把所有回覆合併
messagecombinlist=[" ".join(["".join(meg) for meg in megs if meg!=None and meg!='null'and meg!='']) for megs in dienmessagelist]
messagecombinlist[0]

'感謝分享，也幫我推一個，謝謝http://www.ipeen.com.tw/comment/943950 到台南好像不能錯過安平的豆花耶給你非常實用也希望你能給我實用喔【電影類別 / 動作】玩命關頭7-復仇之路 回歸到起點推~~~同是台南人XD這家真的好吃http://www.ipeen.com.tw/comment/838440麻煩支持 一下喔:) 鍋燒意麵這款中山店沒有賣!!!! 竹炭豆畫...黑色的嘛==?感覺鮮奶的比較好吃去安平，必吃美食~~讚！ 好像我去過這家！很面熟，但是不太確定。 回推囉，豆花看起來不錯呢！ 感覺你們玩超多天的耶XD呵呵 我跟阿左去三天兩夜都感覺沒有你們玩得多呢我喜歡粉圓豆花。這一家商妮去台南玩的時候也有去吃，人好多~ 豆花看起來好好吃><紅豆在上面顏色就看起來很漂亮最近的美食分享，喜歡的話幫忙給個讚吧非常謝謝您GODIVA比利時巧克力●奢華享受http://www.ipeen.com.tw/comment/611570IOU Cafe●甜點美饌 http://www.ipeen.com.tw/comment/611714安平豆花我也超愛~下次再來台南~安平豆花地鍋燒意麵和對面的福記水煎包也都蠻好吃的~~很特別的豆花.看起來很嫩那天下去看到好多人，就沒有去排了!!! 每次去安平 我都必吃那家豆花耶~~~超愛!! 我喜歡粉圓加檸檬，酸酸甜甜又清涼上次去台南很猶豫要吃這間還是黑豆花，結果選了黑豆花太太太銷魂了謝謝分享 超愛珍珠豆花的啦~~~~~~檸檬也蠻好吃的~隔壁鍋燒意麵也好吃 好久沒吃了! 上次很像點珍珠檸檬豆花~好吃也喜歡酸甜感我上次也是點檸檬的耶!  紅豆看起來挺不賴的這不是排隊排慘的那家嗎XD豆花好綿啊~看起來就很好吃可惜我上回去~找不到這家，最後隨便路邊找了一間進去吃，結果難吃到爆~~~我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒出名時，我就常去那吃，當時是家小小的店，店裡桌以才3~4張而已呢檸檬的超好吃!!!他們家的鍋燒意面也很不錯~下次可以試試~!\t※ 引述 ✿花露露✿ 之言：\t檸檬的超好吃!!!\t他們家的鍋燒意面也很不錯~下次可以試試~!蝦密??還有賣鍋燒意麵阿?真的假的去很多次了耶~都沒發現....XD\t※ 引述 品味廣東粥 之言：\t我也超愛這家豆花，只要有到安平我都會來這家，記的以前他們還沒

In [14]:
#把所有評論合併
commentcombinelist = [" ".join([j["content"] for j in i["comment"]]) for i in ipeenjson]
commentcombinelist[0]

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/ 來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜 這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是平日的暑假店裡還是

In [15]:
#合併評論+回覆
pincombinelist=[comment+" "+message for comment,message in zip(commentcombinelist,messagecombinlist)]
pincombinelist[0]

'簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/ 來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜 這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是平日的暑假店裡還是

In [16]:
#清一下記憶體
import gc
gc.collect()

0


# 將許多可能會用來分析的項目做成list

# 做TD-IDF分析

In [17]:

st=time.time()
#取得各店家的斷完詞評論pincutlist
pincutlist = []
cutcount=0
for pincombine in pincombinelist:
    cutcount+=1
    if cutcount%200==0:
        print("以切"+str(cutcount)+"篇文章的詞")
    words = jieba.cut(pincombine, cut_all=False)
    pincut=" ".join([word for word in words if word not in stopwordset and '\u4e00' <= word and word <= '\u9fff'])
    pincutlist.append(pincut)
    
#所有店家的用詞加總前三百wordCountlist300
wordCountlist=[Counter(pincut.split(" ")).most_common(300) for pincut in pincutlist]

#TF計算 (TF=這個字出現的次數/這篇文章的總字數)
#(sum(dict(wordCount).values())=>一篇所有字的字數  wordkv[1]>>該文字字數  wordkv[0]>>該文字)
#TFList每篇文章的詞 的詞頻
TFList=[[((wordkv[0],wordkv[1]/sum(dict(wordCount).values()))) for wordkv in wordCount] for wordCount in wordCountlist]


#全文章共用了幾個詞(set可以去除重複，可迭代)
wordlist=[]
for wordCount in wordCountlist:
    for word in wordCount:
        wordlist.append(word[0])
wordlist=set(wordlist)

#nDien所有的文章(店家)數
ndien=len(dienlist)

#wordappear計算每個字出現於 (幾篇) 文章
wordappear={}
appearcount=0
for word in wordlist:
    appearcount+=1
    if appearcount%1000==0:
        print("以計算"+str(appearcount)+"個文字的出現篇數")
    n=0
    for pincut in pincutlist:
        if word in pincut:
            n+=1
    wordappear[word]=n
    
#IDF(逆向檔案頻率)=某詞  所有文章數/在幾篇文章出現 開log10 >>次方數為所
IDFlist={word:math.log(ndien/wordappear[word],10) for word in wordappear}

#dienTF每家店的全字TF值 wordTF每個詞的TF   wordTF[0]詞 wordTF[1]詞的TF值  IDFlist[wordTF[0]]詞的IDF值
TF_IDFlist=[Counter({wordTF[0]:wordTF[1]*IDFlist[wordTF[0]]for wordTF in dienTF})for dienTF in TFList]
for name,TFIDF in zip(dienlist,TF_IDFlist):
    if name in TFIDF:
        del TFIDF[name]


et=time.time()
print(et-st)

以切200篇文章的詞
以切400篇文章的詞
以切600篇文章的詞
以切800篇文章的詞
以切1000篇文章的詞
以切1200篇文章的詞
以切1400篇文章的詞
以切1600篇文章的詞
以切1800篇文章的詞
以切2000篇文章的詞
以切2200篇文章的詞
以切2400篇文章的詞
以切2600篇文章的詞
以切2800篇文章的詞
以切3000篇文章的詞
以切3200篇文章的詞
以切3400篇文章的詞
以切3600篇文章的詞
以切3800篇文章的詞
以切4000篇文章的詞
以切4200篇文章的詞
以切4400篇文章的詞
以切4600篇文章的詞
以切4800篇文章的詞
以切5000篇文章的詞
以切5200篇文章的詞
以切5400篇文章的詞
以切5600篇文章的詞
以計算1000個文字的出現篇數
以計算2000個文字的出現篇數
以計算3000個文字的出現篇數
以計算4000個文字的出現篇數
以計算5000個文字的出現篇數
以計算6000個文字的出現篇數
以計算7000個文字的出現篇數
以計算8000個文字的出現篇數
以計算9000個文字的出現篇數
以計算10000個文字的出現篇數
以計算11000個文字的出現篇數
以計算12000個文字的出現篇數
以計算13000個文字的出現篇數
以計算14000個文字的出現篇數
以計算15000個文字的出現篇數
以計算16000個文字的出現篇數
以計算17000個文字的出現篇數
以計算18000個文字的出現篇數
以計算19000個文字的出現篇數
以計算20000個文字的出現篇數
以計算21000個文字的出現篇數
以計算22000個文字的出現篇數
以計算23000個文字的出現篇數
以計算24000個文字的出現篇數
以計算25000個文字的出現篇數
以計算26000個文字的出現篇數
以計算27000個文字的出現篇數
以計算28000個文字的出現篇數
以計算29000個文字的出現篇數
以計算30000個文字的出現篇數
以計算31000個文字的出現篇數
以計算32000個文字的出現篇數
以計算33000個文字的出現篇數
以計算34000個文字的出現篇數
以計算35000個文字的出現篇數
以計算36000個文字的出現篇數
以計算37000個文字的出現篇數
以計算38000個文字的出現篇數
以計算39000個文字的出現篇數
以計算40000個文字的出現

# 將上面的斷詞文章combine在一起 做word2vec model

In [18]:


#將所有的斷詞文章combine在一起，變成超大斷詞字串，並存起來
contentcutcombine=" ".join(pincutlist)
with open('./data/TainanContent.txt','w',encoding="utf-8") as w:
    w.write(contentcutcombine)
    
#將存起來的文章用word2vec.Text8Corpus切成有順序的詞彙list(sentences)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.Text8Corpus("./data/TainanContent.txt")

#將有順序的詞彙list丟給word2vec做訓練
model = word2vec.Word2Vec(sentences, size=300,window=6, min_count=4, workers=4,sg=1)
# sentences:當然了，這是要訓練的句子集，沒有他就不用跑了
# size:這表示的是訓練出的詞向量會有幾維
# alpha:機器學習中的學習率，這東西會逐漸收斂到 min_alpha
# sg:sg=1表示採用skip-gram(由本字去測他字 擴散),sg=0表示採用cbow(由他字測本字 集中)
# window:能往左往右看幾個字的意思
# workers:執行緒數目，除非電腦不錯，不然建議別超過 4
# min_count:若這個詞出現的次數小於min_count，那他就不會被視為訓練對象

# Save our model.
model.save("./data/Tainan.model.bin")


2017-09-15 22:17:18,897 : INFO : collecting all words and their counts
2017-09-15 22:17:18,902 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-09-15 22:17:22,196 : INFO : collected 218969 word types from a corpus of 12603200 raw words and 1261 sentences
2017-09-15 22:17:22,196 : INFO : Loading a fresh vocabulary
2017-09-15 22:17:22,559 : INFO : min_count=4 retains 81623 unique words (37% of original 218969, drops 137346)
2017-09-15 22:17:22,559 : INFO : min_count=4 leaves 12402665 word corpus (98% of original 12603200, drops 200535)
2017-09-15 22:17:22,762 : INFO : deleting the raw counts dictionary of 218969 items
2017-09-15 22:17:22,770 : INFO : sample=0.001 downsamples 20 most-common words
2017-09-15 22:17:22,772 : INFO : downsampling leaves estimated 12049013 word corpus (97.1% of prior 12402665)
2017-09-15 22:17:22,772 : INFO : estimated required memory for 81623 words and 300 dimensions: 236706700 bytes
2017-09-15 22:17:23,023 : INFO : resetting la

2017-09-15 22:18:36,441 : INFO : PROGRESS: at 28.95% examples, 240836 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:18:37,465 : INFO : PROGRESS: at 29.36% examples, 240868 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:18:38,514 : INFO : PROGRESS: at 29.79% examples, 240930 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:18:39,530 : INFO : PROGRESS: at 30.21% examples, 241117 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:18:40,580 : INFO : PROGRESS: at 30.61% examples, 240930 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:18:41,607 : INFO : PROGRESS: at 31.02% examples, 240948 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:18:42,624 : INFO : PROGRESS: at 31.42% examples, 240851 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:18:43,704 : INFO : PROGRESS: at 31.86% examples, 240963 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:18:44,735 : INFO : PROGRESS: at 32.29% examples, 241083 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:18:45,753 : INFO : PROGRESS: at 32.70% examples, 241120 wor

2017-09-15 22:19:59,137 : INFO : PROGRESS: at 62.28% examples, 241916 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:20:00,150 : INFO : PROGRESS: at 62.71% examples, 241996 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:20:01,176 : INFO : PROGRESS: at 63.14% examples, 242062 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:20:02,179 : INFO : PROGRESS: at 63.55% examples, 242097 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:20:03,275 : INFO : PROGRESS: at 64.00% examples, 242115 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:20:04,325 : INFO : PROGRESS: at 64.44% examples, 242199 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:20:05,343 : INFO : PROGRESS: at 64.87% examples, 242266 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:20:06,355 : INFO : PROGRESS: at 65.28% examples, 242285 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:20:07,371 : INFO : PROGRESS: at 65.68% examples, 242252 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:20:08,401 : INFO : PROGRESS: at 66.12% examples, 242362 wor

2017-09-15 22:21:21,401 : INFO : PROGRESS: at 95.64% examples, 242753 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:21:22,418 : INFO : PROGRESS: at 96.05% examples, 242761 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:21:23,428 : INFO : PROGRESS: at 96.43% examples, 242694 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:21:24,449 : INFO : PROGRESS: at 96.83% examples, 242659 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:21:25,464 : INFO : PROGRESS: at 97.24% examples, 242669 words/s, in_qsize 7, out_qsize 0
2017-09-15 22:21:26,478 : INFO : PROGRESS: at 97.65% examples, 242674 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:21:27,498 : INFO : PROGRESS: at 98.07% examples, 242687 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:21:28,507 : INFO : PROGRESS: at 98.46% examples, 242667 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:21:29,514 : INFO : PROGRESS: at 98.87% examples, 242685 words/s, in_qsize 8, out_qsize 0
2017-09-15 22:21:30,524 : INFO : PROGRESS: at 99.29% examples, 242691 wor

In [19]:
gc.collect()

0

# 試玩word2vec的model  需解除#

In [20]:
# from gensim.models import word2vec
# from gensim import models
# import logging

# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# model = models.Word2Vec.load('.\data\Tainan.model.bin')

# print("提供 3 種測試模式\n")
# print("輸入一個詞，則去尋找前一百個該詞的相似詞")
# print("輸入兩個詞，則去計算兩個詞的餘弦相似度")
# print("輸入三個詞，進行類比推理")

# while True:
#     try:
#         query = input()
#         q_list = query.split()

#         if len(q_list) == 1:
#             print("相似詞前 30 排序")
#             res = model.most_similar(q_list[0],topn = 30)
#             for item in res:
#                 print(item[0]+","+str(item[1]))

#         elif len(q_list) == 2:
#             print("計算 Cosine 相似度")
#             res = model.similarity(q_list[0],q_list[1])
#             print(res)
#         else:
#             print("%s之於%s，如%s之於" % (q_list[0],q_list[2],q_list[1]))
#             res = model.most_similar([q_list[0],q_list[1]], [q_list[2]], topn= 20)
#             for item in res:
#                 print(item[0]+","+str(item[1]))
#         print("----------------------------")
#     except Exception as e:
#         print(repr(e))


# 試Rocchio、bayes、kmeans、SVM分群 (仿白話大數據寫法)

In [21]:
#取得各店家的斷完詞評論contentcutlist
#店家種類字典stylelist
# from sklearn.datasets import fetch_20newsgroups
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer

#斷詞function
def cut_text_list(textlist):
    textcutlist=[]
    for text in textlist:
        cutwords=jieba.cut(text)
        cuttext=" ".join([word for word in cutwords if word not in stopwordset and '\u4e00' <= word <= '\u9fff'])
        textcutlist.append(cuttext)
    return textcutlist

#文字轉向量矩陣
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(pincutlist)
print('文字轉向量矩陣')

#向量矩陣做tfidf

tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('tfidf')



#ptt測試集|
with open('./data/foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)

#試用Roccio分群
# from sklearn.neighbors.nearest_centroid import NearestCentroid
st=time.time()
clf = NearestCentroid().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Roccio")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)


#試用bayes分群
# from sklearn.naive_bayes import MultinomialNB
st=time.time()
clf = MultinomialNB().fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("bayes")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

#試用kmeans分群
# from sklearn.neighbors import KNeighborsClassifier
st=time.time()
clf = KNeighborsClassifier(15).fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("Kmeans")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

#試用svm分群
# from sklearn import svm
st=time.time()
clf = svm.SVC(kernel="linear").fit(X_train_tfidf,stylelist)
predicted =clf.predict(x_new_tfidf)
print("svm")
pprint.pprint(predicted)
ed=time.time()
print(ed-st)

文字轉向量矩陣
tfidf
Roccio
array(['台菜餐廳', '美式料理', '義式料理', '西點烘焙、麵包', '其他鍋類', '', '臭豆腐', '', '複合式',
       '美式料理', '其他小吃', '泰式料理', '台菜餐廳'],
      dtype='<U13')
0.8050458431243896
bayes
array(['其他小吃', '複合式', '複合式', '複合式', '複合式', '其他小吃', '其他小吃', '其他小吃', '複合式',
       '其他小吃', '其他小吃', '其他小吃', '其他小吃'],
      dtype='<U13')
1.562089204788208
Kmeans
array(['中式', '複合式', '咖啡專賣', '其他小吃', '麵食點心', '休閒零食', '其他小吃', '西點烘焙、麵包',
       '西點烘焙、麵包', '複合式', '中式', '麵食點心', '其他小吃'],
      dtype='<U13')
0.47302722930908203
svm
array(['其他小吃', '複合式', '義式料理', '西點烘焙、麵包', '其他小吃', '其他小吃', '臭豆腐', '其他小吃',
       '複合式', '其他小吃', '其他小吃', '泰式料理', '其他小吃'],
      dtype='<U13')
247.16099214553833


# 自己肉眼辨識的答案是 合菜 美式 義式 西點 鍋類 冰品 臭豆腐 其他 義式 美式 小吃 泰式 台菜

# 最準的是 Roccio > svm > kmeans > bayes

In [22]:
#參考PTT斷詞文章
docs_new

['壹等賞 桃園 龍潭 看板 批 踢踢 實業 坊 批 踢踢 實業 坊 看板 聯絡 資訊 返回 看板 分享 作者 看板 標題 壹等賞 桃園 龍潭 時間 本文 轉錄 看板 作者 看板 標題 壹等賞 桃園 龍潭 時間 圖文 版 今天 提早到 新竹 火車站 附近 髮 瑟 剪 頭髮 整個 人 變得 清爽 免不了 念 幾句 想 以後 頭髮 剪 好 今天 比較 晚 下班 六點 多才 離開 公司 交通 正 處於 尖峰 時刻 光復路 繞 一圈 還了 漫畫 之後 決定 晚上 吃 桃園 龍潭 壹等賞 新竹 文化 街上 紅葉 告訴 目前 肚子 不餓 想 說 現在 開 壹等賞 時間 應該 夠長 肚子 應該 餓 決定 上次 曾經 吃過 一次 壹等賞 享用 晚餐 壹等賞 位於 龍潭 渴望 園區 附近 上次 來過 一次 不知 道路 走 知道 高速公路 累得 睡著 開 十分鐘 下 關西 交 流道 辦法 睡 一下 一下 交流 道 吵醒 請 幫 帶路 今 天 再 走過 一次 不太 認得 路 晚上 天色 黑 另一方 面是 山路 白天 去過 難記 住 路 走 下面 名片 背後 地圖 有興 趣 參考 一下 地址 桃園縣 龍潭 鄉 高原 村 中原 路 弄 電話 營業時間 每週 公休 今天 一到 停車場 滿滿的 車子 已經 七點 多了還 一堆 人 一看 發現 應該 公司 聚餐 很多人 穿著 同樣 制服 外面 抽 飯後 煙 空間 很大 晚到 位置 坐 壹等賞 一個 茶園 旁邊 一片 很大 茶 田 裡面 餐廳 賣 茶葉 東西 就連 洗手間 洗手 乳 茶葉 香味 服務 人員 一間 廂房 裡面 已經 坐著 滿滿的 客人 一到 看到 客人 本來 已經 心理準備 廚房 上菜 比較慢 一點 一點 完菜 不到 三分鐘 菜 一道 一道 送上來 真 佩服 廚房 裡面 大廚 燒菜 功力 上菜 食物 好吃 熱茶 免費 餐桌上 一壺 大壺 熱茶 免費 提供 飲用 醃菜 頭 免費 很喜歡 吃 只 吃 兩塊 剩下 吃 光光 用來 搭配 稀飯 一起 吃 最好 稀飯 點 好吃 茶油 拌 飯 珍香蜜 茶 鵝 小 茶鵝 很好吃 有點 貴 外面 賣 鵝肉 不太 應該 有用 茶葉 燻 很香 養生茶 油拌 飯 點了 兩碗 四碗 普通 碗 份量 要點 茶油 拌 飯 吃 哪有 吃白飯 道理 桂花 蜜 排骨 小 糖醋排骨 吃 覺得 還不錯 吃 最後 有點 吃 不

In [23]:
dienallpinlist[0]

['簡單、傳統的安平豆花，純白的豆花特別的挑逗我們的胃口。安平，兼具了台南傳統與現代的一個區域，而作為安平代表的傳統美食，一如我們所知的就是安平豆花與周氏蝦捲了。今天，帶點炎熱的天氣，是來點清涼降火冰品的好日子，當然是來喊個豆花消暑了，而吃安平豆花當然是要去最著名的同記安平豆花。同記安平豆花光是在安平就有兩個店面，相距只有一公里左右，今天我們來到的是總店。雖然是平日下午，但店內一副忙碌的景象。剛有客人離開，趕緊搶佔一桌，店內用的是原木的大張長桌以及長椅，很有古風呢。牆上擺的是這些年來到過安平豆花的名人，真的是不少。同記安平豆花在安平已經有四十多年的歷史了，取名「同」記是因為最早的創始人黃慶同名字中有個同字。從小推車一路叫賣開始，到現在成為台南十大傳統名產之一，其中艱辛不足與外人道(所以我們當然不知道了，跳過)；從1993年成立店面後，原本大家稱呼的安平豆花也正式的掛上了「同記」的前綴了。不過雖然安平區的豆花店逐漸變多，但是大家只要說到安平豆花，就知道是指同記安平豆花說。快快來點豆花了，種類並不是非常多，但是道道都是經典。我們選了最正宗、最傳統的紅豆豆花跟綠豆豆花各一碗。一切依古法精製，非基因改造有機黃豆製作，光看成色就令人心動。紅豆部分，則是由來自屏東萬丹的極品紅豆，經過數小時的熬煮後，以最佳的紅豆香氣呈現在我們眼前。好吃，加點一碗紅豆豆花，每顆紅豆都保持著完整的外觀，入口都很有口感說。每一口豆花的口感都是軟嫩細緻，入口傳統的黃豆香氣，手工製作跟用洋菜粉勾芡出來的口感就是不一樣。另外綠豆豆花則除了軟透心的綠豆外，搭著嫩豆花以及微甜的甘蔗糖水一起入口，高雅芬芳得猶如在咖啡館享用下午茶。好吃的豆花，來到安平千萬不要錯過，在老街逛累了，順便來碗冰涼透心的豆花，真是大享受啊！同記安平豆花(總店)(09:00~23:00)台南市安平區安北路433號(06-3915385‧3915057)http://www.tongji.com.tw/',
 '來到台南就來試試這家老字號豆花店古早味啊非常有名的一家店裡面人也很多但我不懂哈哈我覺得吃起來跟其他店沒甚麼差別拉可能又是我舌頭頓吧老字號自然歷史悠久牆上掛滿了名人照可以家不同料配豆花我們點了紅豆小珍珠綠豆豆花很軟入口即化湯特別甜',
 '這家是在地人推薦的安平豆花喔老字號豆花當然要來嘗試ㄧ下阿在這炎熱的天氣裡最需要來ㄧ碗甜品了阿這天是

In [24]:
gc.collect()

0

# 試者用自己的字典給店家分數(一篇評論的正負評價詞最多算5個)

In [25]:
#各店家評論diencommentlist
#取得各店家的斷完詞評論contentcutlist
#自製餐廳相關字字典pinlist
#pindict每個字的評分dict
#各店家評論diencommentlist

#pinCountMaxper5list 一個店家出現的餐廳相關詞數量(一個篇評論的相同詞最多給5個分數)
st=time.time()
pinCountMaxper5list = []
countN=0
for dienallpin in dienallpinlist:
    dienWordcount={}
    for allpin in  dienallpin:
        countN+=1
        if countN%100==0:
            print("已算完"+str(countN)+"篇情緒字")
        for pinword in pinlist:
            if len(re.findall(pinword,allpin))>0:
                if len(re.findall(pinword,allpin))>5:
                    npinword=5
                else:
                    npinword=len(re.findall(pinword,allpin))
                if pinword not in dienWordcount:
                    dienWordcount[pinword]=0
                dienWordcount[pinword]+=npinword
    pinCountMaxper5list.append(dienWordcount)
ed=time.time()
print(ed-st)

已算完100篇情緒字
已算完200篇情緒字
已算完300篇情緒字
已算完400篇情緒字
已算完500篇情緒字
已算完600篇情緒字
已算完700篇情緒字
已算完800篇情緒字
已算完900篇情緒字
已算完1000篇情緒字
已算完1100篇情緒字
已算完1200篇情緒字
已算完1300篇情緒字
已算完1400篇情緒字
已算完1500篇情緒字
已算完1600篇情緒字
已算完1700篇情緒字
已算完1800篇情緒字
已算完1900篇情緒字
已算完2000篇情緒字
已算完2100篇情緒字
已算完2200篇情緒字
已算完2300篇情緒字
已算完2400篇情緒字
已算完2500篇情緒字
已算完2600篇情緒字
已算完2700篇情緒字
已算完2800篇情緒字
已算完2900篇情緒字
已算完3000篇情緒字
已算完3100篇情緒字
已算完3200篇情緒字
已算完3300篇情緒字
已算完3400篇情緒字
已算完3500篇情緒字
已算完3600篇情緒字
已算完3700篇情緒字
已算完3800篇情緒字
已算完3900篇情緒字
已算完4000篇情緒字
已算完4100篇情緒字
已算完4200篇情緒字
已算完4300篇情緒字
已算完4400篇情緒字
已算完4500篇情緒字
已算完4600篇情緒字
已算完4700篇情緒字
已算完4800篇情緒字
已算完4900篇情緒字
已算完5000篇情緒字
已算完5100篇情緒字
已算完5200篇情緒字
已算完5300篇情緒字
已算完5400篇情緒字
已算完5500篇情緒字
已算完5600篇情緒字
已算完5700篇情緒字
已算完5800篇情緒字
已算完5900篇情緒字
已算完6000篇情緒字
已算完6100篇情緒字
已算完6200篇情緒字
已算完6300篇情緒字
已算完6400篇情緒字
已算完6500篇情緒字
已算完6600篇情緒字
已算完6700篇情緒字
已算完6800篇情緒字
已算完6900篇情緒字
已算完7000篇情緒字
已算完7100篇情緒字
已算完7200篇情緒字
已算完7300篇情緒字
已算完7400篇情緒字
已算完7500篇情緒字
已算完7600篇情緒字
已算完7700篇情緒字
已算完7800篇情緒字
已算完7900篇情緒字
已算完8000篇情緒字
已算完8100篇情緒字
已算完8200篇情緒字
已算完8300篇情緒字
已算完8400篇情緒字
已

In [26]:
pinCountMaxper5list[50]

{'': 432,
 'Q彈': 1,
 '一品': 3,
 '一般般': 1,
 '不太喜歡': 2,
 '不好吃': 2,
 '不愛': 4,
 '不膩口': 4,
 '也不錯': 8,
 '乾了點': 1,
 '乾淨': 5,
 '亂': 5,
 '享受': 9,
 '便宜': 10,
 '偏酸': 1,
 '健康': 9,
 '入口即化': 1,
 '划算': 5,
 '剛好': 16,
 '口感紮實': 1,
 '古色古香': 1,
 '可愛': 20,
 '吃飽': 1,
 '合理': 2,
 '喜歡': 45,
 '大份': 2,
 '大碗': 3,
 '天然': 8,
 '天然食材': 2,
 '天龍': 4,
 '太多了': 3,
 '太甜': 14,
 '太臭': 1,
 '好吃': 96,
 '好喝': 4,
 '好好吃': 12,
 '好有味道': 2,
 '好甜': 3,
 '好貴': 3,
 '好食': 1,
 '實惠': 1,
 '寬敞': 1,
 '小巧': 4,
 '小貴': 1,
 '平價': 3,
 '平心而論': 1,
 '很不錯': 3,
 '很便宜': 3,
 '很喜歡': 7,
 '很好吃': 16,
 '很有禮貌': 1,
 '很棒': 1,
 '很清爽': 2,
 '很濃郁': 10,
 '很甜': 1,
 '很美味': 1,
 '很脆': 1,
 '很膩': 1,
 '很難吃': 1,
 '很香': 4,
 '後悔': 2,
 '態度好': 1,
 '整齊': 4,
 '新鮮': 11,
 '明亮': 5,
 '有禮貌': 1,
 '有落差': 1,
 '有點小': 2,
 '有點硬': 3,
 '柔軟': 3,
 '浪漫': 2,
 '清幽': 1,
 '清爽': 16,
 '溫暖': 3,
 '滿喜歡': 1,
 '滿意': 1,
 '漂亮': 4,
 '漲成': 1,
 '濃郁': 42,
 '熱呼呼的': 1,
 '熱情': 6,
 '爽口': 1,
 '特別': 21,
 '甜度恰到好處': 2,
 '精緻': 6,
 '精美': 3,
 '經濟實惠': 1,
 '綿密': 21,
 '綿密口感': 1,
 '綿綿的': 2,
 '缺陷': 1,
 '美味': 16,
 '脆脆的': 3,
 '膩口': 

In [27]:
#店家評論總數Ncommentlist
#店家回覆總數Nmessage
#將六篇回覆視為一篇文章，做正規化(防止評論數過多的店家分數極端化)
Npartpin=[int(Nmessage/6)+Ncomment for Nmessage,Ncomment in zip(Nmessagelist,Ncommentlist)]


#大致計算每家店的分數dienscorelist
dienscorelist=[]
for pinCount,N in zip(pinCountMaxper5list,Npartpin):
    dienscore=0
    for pin in pinCount:
        dienscore+=pinCount[pin]*pindict[pin]/N
    dienscorelist.append(dienscore)

In [28]:
dienscorelist[2]

8.74269662921348

In [29]:
#mydict為自己的字典的資料
#格式為:
######否開頭tag 為 非否tag 反義
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予正分數 tag評價詞(非否評價) 同意字群同意字群同意字群同意字群同意字群同意字群
#給予負分數 tag評價詞(否開頭評價) 同意字群同意字群同意字群同意字群同意字群同意字群

#line.split(",")[1]為tag!!! word為與tag對應的同義字
synonyms=[{word:line.split(",")[1] for word in line.split(",")[1:]} for line in mydict.split("\n") if line.split(",")[0]!=""]

pinsynonymsdict={}
for synonym in synonyms:
    pinsynonymsdict.update(synonym)
del pinsynonymsdict[""]




In [30]:
pinsynonymsdict

{'CP值很低': '否划算',
 'CP值非常高': '划算',
 'CP值高份量夠': '划算',
 'Q彈': '食物美味',
 'Q彈柔嫩': '食物美味',
 'cp值不高': '否划算',
 'cp值太低': '否划算',
 'cp值很高': '划算',
 'cp值高': '划算',
 'hen好吃': '食物美味',
 'juicy': '食物美味',
 '一個人吃剛剛好': '小份量',
 '一口想把它喝光的衝動': '食物美味',
 '一品': '食物美味',
 '一問三不知': '否服務好',
 '一塵不染': '環境好',
 '一定要稱讚一下': '食物美味',
 '一樣無敵': '食物美味',
 '一致好評': '食物美味',
 '一般般': '否食物美味',
 '不乾不柴': '食物美味',
 '不乾柴': '食物美味',
 '不乾淨': '否環境好',
 '不划算': '否划算',
 '不卡牙': '食物美味',
 '不可多得': '服務好',
 '不夠酥脆': '否食物美味',
 '不太優': '否食物美味',
 '不太喜歡': '否食物美味',
 '不太好吃': '否食物美味',
 '不太愛吃': '否食物美味',
 '不太慣': '否食物美味',
 '不好吃': '否食物美味',
 '不對味': '否食物美味',
 '不怎麼樣': '否食物美味',
 '不想離開': '其他正面情緒',
 '不愛': '否食物美味',
 '不愛吃': '否食物美味',
 '不愛它': '否食物美味',
 '不搭': '否食物美味',
 '不敢恭維': '否環境好',
 '不斐': '否划算',
 '不新鮮': '否食物美味',
 '不會再來': '否其他正面情緒',
 '不會去第二次': '否其他正面情緒',
 '不會吃不起': '划算',
 '不會太重': '口味清淡',
 '不會太鹹': '口味清淡',
 '不會過於甜膩': '食物美味',
 '不會過甜': '食物美味',
 '不會過膩': '食物美味',
 '不油膩': '食物美味',
 '不熱': '否食物美味',
 '不用擔心': '其他正面情緒',
 '不用贅述就是好吃阿': '食物美味',
 '不算便宜': '否划算',
 '不脆': '否食物美味',
 '不膩口': '食物美味',


In [31]:
#每篇文章pinCountMaxper5list評價詞出現次數
#評價同義詞pinsynonymsdict

#取得各餐廳的評價特徵分數featurescores!!!
featurescores=[]

#將評價詞加總及評論次數做iterate
for pinCountMaxper5,N in zip(pinCountMaxper5list,Npartpin):
    dienfeaturescore={}
    
    # 評價詞出現次數/評論次數 做為分數，其中否字開頭給負分
    for pin in pinCountMaxper5:
        if pin !="":
            if pinsynonymsdict[pin][0]=="否":
                if pinsynonymsdict[pin][1:] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin][1:]]=0
                dienfeaturescore[pinsynonymsdict[pin][1:]]-=round(pinCountMaxper5[pin]/N,2)*1
            else:
                if pinsynonymsdict[pin] not in dienfeaturescore:
                    dienfeaturescore[pinsynonymsdict[pin]]=0
                dienfeaturescore[pinsynonymsdict[pin]]+=round(pinCountMaxper5[pin]/N,2)
    if len(dienfeaturescore)==0:
        dienfeaturescore["無評"]=1
    featurescores.append(dienfeaturescore)
    
    
    

# 將上述屬性結合成一個大分析表

In [32]:
BigAnalyzeTable=[]
for name,style,Ncomment,contentcut,wordCount,TFIDF,pinCountMax5,score,featurescore,introduction,coordinate in zip(dienlist,stylelist,Npartpin,pincutlist,wordCountlist,TF_IDFlist,pinCountMaxper5list,dienscorelist,featurescores,introductionlist,coordinatelist):
    dien={}
    dien['name']=name
    dien['style']=style
    dien['Ncomment']=Ncomment
    dien['contentcut']=contentcut
    dien['wordCount']=wordCount
    dien['TF_IDF']=TFIDF
    dien['pinCountMaxper5']=pinCountMax5
    dien['score']=score
    dien['featurescores']=featurescore
    dien['tags']=[tag[0] for tag in Counter(TFIDF).most_common(5)]
    
    dien['introduction']=introduction
    dien['coordinate']=coordinate
    
    
    BigAnalyzeTable.append(dien)


# dienlist
# stylelist
# Ncommentlist
# pincutlist
# wordCountlist
# TF_IDFlist
# pinCountMaxper5list
# dienscorelist
# featurescores


#店家種類字典introductionlist
# introductionlist = [dien["introduction"] for dien in ipeenjson]

#店家種類字典coordinatelist
# coordinatelist = [dien['coordinate'] for dien in ipeenjson]

In [33]:
gc.collect()

0

# 將每個自定義的評價詞做正規化評分、並且將分數達標者做為標籤貼上!

In [34]:
#算出每個評論的評論數N時的 最大值b 最小值g for正規化


#allpin=評論詞list
allpin=set([line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"])
standard={}

#standard為正規化所需用的值
for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
        get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
        get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
        standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

服務好+0	3.0	-3.0
服務好+5	1.9	-0.5
服務好+10	1.0	-0.2
服務好+15	0.9	-0.2
偏甜+0	5.0	0
偏甜+5	1.3	0
偏甜+10	1.1	0
偏甜+15	1.7	0
划算+0	14.0	-7.0
划算+5	5.5	-4.9
划算+10	5.1	-1.5
划算+15	2.5	-1.6
口味清淡+0	3.0	0
口味清淡+5	0.8	0
口味清淡+10	0.5	0
口味清淡+15	1.1	0
份量大+0	9.0	0
份量大+5	2.7	0
份量大+10	3.5	0
份量大+15	3.1	0
環境好+0	26.0	-3.0
環境好+5	6.7	-0.6
環境好+10	6.1	-0.5
環境好+15	5.3	-0.3
食物很香+0	8.0	0
食物很香+5	3.7	0
食物很香+10	2.3	0
食物很香+15	4.2	0
偏酸+0	1.0	0
偏酸+5	0.5	0
偏酸+10	0.3	0
偏酸+15	0.4	0
偏辣+0	5.0	0
偏辣+5	1.3	0
偏辣+10	0.9	0
偏辣+15	1.3	0
食物美味+0	37.0	-19.0
食物美味+5	23.0	0
食物美味+10	19.9	0
食物美味+15	21.6	0
食物精緻漂亮+0	15.0	0
食物精緻漂亮+5	7.2	0
食物精緻漂亮+10	4.5	0
食物精緻漂亮+15	6.0	0
其他正面情緒+0	18.0	-2.0
其他正面情緒+5	7.5	-0.3
其他正面情緒+10	9.7	0
其他正面情緒+15	8.1	0
重口味+0	5.0	0
重口味+5	1.9	0
重口味+10	1.3	0
重口味+15	1.3	0
小份量+0	5.0	0
小份量+5	2.4	0
小份量+10	1.7	0
小份量+15	1.5	0


In [35]:
standard

{'份量大0': (4.5, 4.5),
 '份量大10': (1.75, 1.75),
 '份量大15': (1.55, 1.55),
 '份量大5': (1.35, 1.35),
 '偏甜0': (2.5, 2.5),
 '偏甜10': (0.55, 0.55),
 '偏甜15': (0.85, 0.85),
 '偏甜5': (0.65, 0.65),
 '偏辣0': (2.5, 2.5),
 '偏辣10': (0.45, 0.45),
 '偏辣15': (0.65, 0.65),
 '偏辣5': (0.65, 0.65),
 '偏酸0': (0.5, 0.5),
 '偏酸10': (0.15, 0.15),
 '偏酸15': (0.2, 0.2),
 '偏酸5': (0.25, 0.25),
 '其他正面情緒0': (8.0, 10.0),
 '其他正面情緒10': (4.85, 4.85),
 '其他正面情緒15': (4.05, 4.05),
 '其他正面情緒5': (3.6, 3.9),
 '划算0': (3.5, 10.5),
 '划算10': (1.7999999999999998, 3.3),
 '划算15': (0.44999999999999996, 2.05),
 '划算5': (0.2999999999999998, 5.2),
 '口味清淡0': (1.5, 1.5),
 '口味清淡10': (0.25, 0.25),
 '口味清淡15': (0.55, 0.55),
 '口味清淡5': (0.4, 0.4),
 '小份量0': (2.5, 2.5),
 '小份量10': (0.85, 0.85),
 '小份量15': (0.75, 0.75),
 '小份量5': (1.2, 1.2),
 '服務好0': (0.0, 3.0),
 '服務好10': (0.4, 0.6),
 '服務好15': (0.35, 0.55),
 '服務好5': (0.7, 1.2),
 '環境好0': (11.5, 14.5),
 '環境好10': (2.8, 3.3),
 '環境好15': (2.5, 2.8),
 '環境好5': (3.0500000000000003, 3.65),
 '重口味0': (2.5, 2.5),
 '重口味10': (0.65,

In [36]:
#進行正規化的處理
for dien in BigAnalyzeTable:
    N=dien['Ncomment']
    for feature in dien['featurescores']:
        if feature !="無評":
            if N<6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(0)][1],2)
            elif N<11 and N>=6:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(5)][1],2)
            elif N<16 and N>=11:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(10)][1],2)
            else:
                dien['featurescores'][feature]=round((dien['featurescores'][feature])/standard[feature+str(15)][1],2)

# 正規化後，每個屬性的評價區間都為2! 但未避免有些沒被負評的店判斷成負評，不將中間值移至0的位置

In [37]:


allpin=[line.split(",")[1] for line in mydict.split("\n") if line.split(",")[0]!="" and line.split(",")[1][0]!="否"]
standard={}

for pin in allpin:
    for rang in [(0,5),(5,10),(10,15),(15,100)]:
#         get={}
        values=[dien['featurescores'][pin] for dien in BigAnalyzeTable if pin in dien['featurescores'] and dien["Ncomment"]>rang[0] and dien["Ncomment"]<=rang[1]]
        b=0
        g=0
        for value in values:
            if value <b:
                b=round(value,1)
            if value >g:
                g=round(value,1)
#         get[pin+str(rang[0])]=((g+b)/2,g-(g+b)/2)
#         standard.update(get)
        print(pin+"+"+str(rang[0])+"\t"+str(g)+"\t"+str(b))

食物美味+0	1.3	-0.7
食物美味+5	2.0	0
食物美味+10	2.0	0
食物美味+15	2.0	0
食物美味+0	1.3	-0.7
食物美味+5	2.0	0
食物美味+10	2.0	0
食物美味+15	2.0	0
划算+0	1.3	-0.7
划算+5	1.1	-0.9
划算+10	1.5	-0.4
划算+15	1.2	-0.8
環境好+0	1.8	-0.2
環境好+5	1.8	-0.1
環境好+10	1.9	-0.1
環境好+15	1.9	-0.1
服務好+0	1.0	-1.0
服務好+5	1.6	-0.4
服務好+10	1.6	-0.4
服務好+15	1.6	-0.3
食物精緻漂亮+0	2.0	0
食物精緻漂亮+5	2.0	0
食物精緻漂亮+10	2.0	0
食物精緻漂亮+15	2.0	0
食物很香+0	2.0	0
食物很香+5	2.0	0
食物很香+10	2.0	0
食物很香+15	2.0	0
其他正面情緒+0	1.8	-0.2
其他正面情緒+5	1.9	-0.1
其他正面情緒+10	2.0	0
其他正面情緒+15	2.0	0
份量大+0	2.0	0
份量大+5	2.0	0
份量大+10	2.0	0
份量大+15	2.0	0
小份量+0	2.0	0
小份量+5	2.0	0
小份量+10	2.0	0
小份量+15	2.0	0
重口味+0	2.0	0
重口味+5	2.0	0
重口味+10	2.0	0
重口味+15	1.9	0
口味清淡+0	2.0	0
口味清淡+5	2.1	0
口味清淡+10	2.2	0
口味清淡+15	2.1	0
偏辣+0	2.0	0
偏辣+5	2.0	0
偏辣+10	1.9	0
偏辣+15	1.9	0
偏甜+0	2.0	0
偏甜+5	2.1	0
偏甜+10	2.0	0
偏甜+15	2.0	0
偏酸+0	2.0	0
偏酸+5	2.0	0
偏酸+10	2.1	0
偏酸+15	1.9	0


# 貼餐廳評價標籤!

In [38]:
pipin=[('食物美味','食物不好吃',"非常好吃",'非常不好吃'),('划算','價格較高昂',"非常便宜",'非常貴')
       ,('環境好','環境比較不好',"環境超棒",'環境很差'),('服務好','服務較差',"服務一級棒",'服務很差')
       ,('其他正面情緒','評論有些負面情緒詞',"網路評論正面情緒非常多",'網路評論負面情緒很多')]
moods=[mood[0] for mood in pipin]
for dien in BigAnalyzeTable:
    newtag=[]
    for feature in dien["featurescores"]:
        if feature in moods:
            if dien["featurescores"][feature]>1:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[2])
            elif dien["featurescores"][feature]>0.4:
                newtag.append(feature)
            elif dien["featurescores"][feature]<-0.5:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[3])
            elif dien["featurescores"][feature]<0.0:
                for pinpare in pipin:
                    if feature==pinpare[0]:
                        newtag.append(pinpare[1])
        else:
            if dien["featurescores"][feature]>0.7:
                newtag.append(feature)
                    
                    
#用set去除重複標籤
    dien['tags']=list(set(dien['tags']+newtag))
    print(dien['name'])
#     print(newtag)
    dien['tags'].append(dien['style'])
    print(dien['tags'])

同記安平豆花(安平總店)
['豆花', '服務較差', '安平', '粉圓', '珍珠', '紅豆', '剉冰、豆花']
深藍咖啡館
['栗子', '蛋糕', '深藍', '千層', '食物美味', '非常貴', '派', '西點烘焙、麵包']
阿堂鹹粥
['虱目', '阿堂', '粥', '食物美味', '魚腸', '鹹粥', '價格較高昂', '其他小吃']
富盛號碗粿
['永樂', '碗粿', '安平', '食物美味', '民族', '魚羹', '其他小吃']
依蕾特布丁奶酪(安平門市)
['布丁', '抹茶', '典藏', '食物美味', '奶酪', '依蕾特', '西點烘焙、麵包']
食下有約 想法廚房
['食物很香', '環境好', '蹄膀', '區郡', '食下有約', '非常好吃', '食物精緻漂亮', '其他正面情緒', '花雕', '想法廚房', '其他異國料理']
飛饗創意西式料理
['服務好', '環境超棒', '燉', '偏酸', '土壤', '沙拉', '典藏', '非常好吃', '食物精緻漂亮', '饗', '其他正面情緒', '義式料理']
莉莉水果店
['水果店', '水果', '肉圓', '前路', '食物美味', '莉莉', '價格較高昂', '剉冰、豆花']
連得堂餅舖
['連得堂', '崇安', '兩包', '煎餅', '餅家', '價格較高昂', '休閒零食']
矮仔成蝦仁飯
['蝦仁飯', '鴨蛋', '蛋湯', '蝦仁', '食物美味', '矮仔成', '價格較高昂', '其他小吃']
泰成水果店
['哈密瓜', '哈蜜瓜', '食物美味', '正興', '芒果', '瓜瓜', '價格較高昂', '果汁']
騷烤家(前鋒總店)
['油條', '燒烤', '騷烤', '烤', '非常好吃', '其他正面情緒', '甜不辣', '炭烤串燒']
阿松割包
['花生醬', '豬舌包', '食物美味', '豬舌', '割包', '瘦肉', '價格較高昂', '麵食點心']
秉醇烘焙坊
['餅乾', '伯爵', '奶酥', '食物很香', '瓜子', '火山', '食物美味', '食物精緻漂亮', '其他正面情緒', '西點烘焙、麵包']
冰鄉
['牛乳', '冰', '草莓', '食物美味', '芒果', '民生路', '剉冰、

['煎餃', '鐵鍋', '食札', '餃子', '食物美味', '担担', '其他日式料理']
銀波布丁(府前老店)
['焦糖', '銀波', '雞蛋', '布丁', '食物美味', '老店', '休閒零食']
性格せいかく 早午餐
['服務一級棒', '划算', '鱷魚', '熱壓', '早午餐', '性格', '拉花', '食物美味', '食物精緻漂亮', '西式早餐']
陶板屋和風創作料理(忠義店)
['陶', '陶板', '食物美味', '其他正面情緒', '蒲小英', '板屋', '飯糰', '綜合日式料理']
旭屋牛丼專賣
['丼', '旭屋', '蔥花', '食物美味', '蛋黃', '重口味', '牛', '其他日式料理']
MissBanana巴娜娜的早午茶趣
['娜娜', '服務好', '環境超棒', '茶趣', '薄餅', '法式', '食物美味', '食物精緻漂亮', '其他正面情緒', '建業', '法式料理']
格那修ganache手作蛋糕
['偏甜', '巧克力', '蛋糕', '環境好', '懷恩街', '食物美味', '太妃糖', '甜點', '食物精緻漂亮', '其他正面情緒', '西點烘焙、麵包']
慕紅豆
['紅豆湯', '大可', '柴燒', '雪綿冰', '紅豆', '飲料專賣店']
愛夏子あいさつ
['服務好', '廣島', '大阪', '環境好', '愛夏子', '食物美味', '食物精緻漂亮', '其他正面情緒', '重口味', '文字', '燒', '鐵板燒']
ChuChu Pâtisserie 法式甜點
['服務好', '偏甜', '環境超棒', '啾', '法式', '蛋糕', '食物很香', '非常好吃', '甜點', '磅', '食物精緻漂亮', '其他正面情緒', '價格較高昂', '西點烘焙、麵包']
阿卿傳統飲品‧冰品
['服務好', '綠豆', '芋頭', '杏仁茶', '食物美味', '阿卿', '饌', '飲料專賣店']
Sunny Pasta陽光義式廚坊
['服務好', '陽光', '環境超棒', '筆管', '非常便宜', '義式', '廚坊', '非常好吃', '食物精緻漂亮', '其他正面情緒', '義大利', '義式料理']
樂禾田
['日夕', '環境好', '早午餐', '安平

杏本善
['服務一級棒', '杏仁茶', '迪利', '褔', '紅杏初', '食物美味', '食物精緻漂亮', '訂票', '飲料專賣店']
麗都大冰果
['冰果', '麗都', '份量大', '冰', '十路', '芒果', '價格較高昂', '果汁']
清閣水樓台南旗艦店(崇明總店)
['湯包', '灌', '環境好', '食物美味', '小英', '水樓', '清閣', '價格較高昂', '麵食點心']
鯨吞燒-精肉食堂(海安店)
['丼', '精肉', '牛排', '鯨吞', '黑豚', '日式燒肉']
Takeat 輕享原食
['輕享', '網路評論正面情緒非常多', '份量大', '食物很香', '環境好', '偏辣', '原食', '餐盒', '沙拉', '非常好吃', '貝果', '小份量', '其他美食']
來享鮮餐廳
['服務一級棒', '食物很香', '重口味', '非常好吃', '其他正面情緒', '享鮮', '芒果', '玉井', '宮保', '老闆娘', '海鮮餐廳']
阿元麵館
['划算', '阿元', '餛飩', '牛肉麵', '食物美味', '重口味', '麻醬', '添福', '麵食點心']
藍屋日本料理(南紡夢時代)
['偏甜', '御膳', '日本料理', '藍屋', '南紡夢', '八品', '食物美味', '綜合日式料理']
ZiZA咖啡
['咖啡', '價格較高昂', '小鳳', '其他正面情緒', '集采', '白鷺', '北路', '義式料理']
滿福亭 まんぷくてい
['茶泡飯', '燒肉飯', '三倍', '滿福亭', '南蠻', '其他日式料理']
豆腐花 DofuWhat
['五十', '豆漿', '鳥媽', '豆腐花', '豆花', '剉冰、豆花']
ICHIRO，ㄧ起囉！
['薯片', '鈴木', '鹹酥雞', '食物很香', '環境好', '一朗', '食物美味', '甜不辣', '鹹酥雞、炸雞排']
Do - Li  BAKERY CAFE 多麗咖啡烘焙美食
['蛋糕', '烘焙', '蒙地卡羅', '食物美味', '多麗', '咖啡', '複合式']
饕公麻辣燙
['麻辣', '饕公', '食物美味', '麻辣鍋', '頭髮', '龍骨', '麻辣火鍋']
福來伯臭豆腐
['順天', '印刷廠',

['兔媽', '兔哥', '上品', '豬排', '狡兔', '鐵板燒']
古典豆豆
['沙朗心', '書局', '升學', '沙瓦', '食物美味', '三極', '義式料理']
立德大學男宿早餐店
['早餐', '捲', '卡拉', '男宿', '立德', '其他小吃']
The Winter 冬風茶飲(大同店)
['冬瓜茶', '鮮奶', '冬瓜', '狡兔', '冬風', '飲料專賣店']
老黃陽春麵
['圓環', '老黃', '食物美味', '肉粽', '麻醬', '體育場', '麵食點心']
小太陽義大利麵(東寧店)
['焗', '筆管', '麵', '白醬', '義大利', '義式料理']
木公生活洋行
['松岳', '馬來西亞', '善生', '安平', '木公', '價格較高昂', '星馬料理']
小妞炒飯
['勝利路', '環境比較不好', '放錢', '炒飯', '宮保雞', '小妞', '熱炒、快炒']
立豐餅店
['大立', '膨鬆劑', '椪', '月子', '碰餅', '西點烘焙、麵包']
異人館無國界料理(市府店)
['壽喜燒', '異人館', '焦小糖', '義大利', '餐點', '咖啡專賣']
二空 村長涼麵
['涼', '賣涼', '村長', '二空', '價格較高昂', '涼皮', '服務較差', '麵食點心']
麥當勞(台南中華店)
['一怒之下', '蘋果派', '麥克', '香橙', '派', '漢堡、炸雞']
ㄔ鍋了 創意鍋物
['海鮮', '龍蝦', '火鍋料', '蒸籠', '鍋物', '其他鍋類']
一等涼冬瓜茶
['善化', '偏甜', '一等', '冬瓜茶', '環境比較不好', '偏辣', '冬瓜', '食物美味', '價格較高昂', '慶安', '飲料專賣店']
微光 somelight branch
['微光', '會員制', '環境好', '調酒', '葡萄酒', '波爾多', '酒吧 Lounge Bar']
成大綠豆湯
['考試', '綠豆', '綠豆湯', '沒誤', '且邊', '飲料專賣店']
湯勺 Soup Ladle
['湯勺', '路西', '食物美味', '羅宋湯', '勝利', '湯品', '其它中式料理']
周記牛肉麵
['周伯通', '牛肉麵', '蒲牢', '周記', '乾

['牛蒡', '酸菜', '蝦醬', '紹子', '合菜', '四川菜']
品廚牛排(新營店)
['阿靜', '划算', '執迷', '品廚', '引述', '牛排', '美式料理']
喜得炭烤三明治
['夏天', '隨拍', '小雞', '意大利', '喜得', '義式料理']
北京麻辣鴛鴦火鍋(健康店)
['鍋底', '鴨血', '星期天', '鴛鴦鍋', '北京', '麻辣火鍋']
海豊亭丼の專門店
['丼', '美學', '豊', '公車', '小編', '居酒屋']
51 STREET 日式炸豬排 和洋料理
['五十一', '服務較差', '豬排', '唐揚', '義大利', '倫子', '日式豬排專賣']
九州麵屋-深夜食堂
['拉麵', '食堂', '九州', '狡兔', '叉燒', '其他日式料理']
韓湘辣年糕(崇德店)
['手搖', '烤肉', '年糕', '韓湘', '炒年糕', '其他異國料理']
咕咕布朗早午餐
['歸仁', '秘食', '食物很香', '早午餐', '食物美味', '貝果', '森林', '價格較高昂', '西式早餐']
哈拉里咖啡專櫃
['有梅汁', '哈拉', '千層', '里', '咖啡', '咖啡專賣']
85度C咖啡蛋糕烘焙專賣店(永康崑山店)
['筱倩', '醫院', '老目', '襁褓', '母親節', '複合式']
一流料理人御弁當
['一流', '微波', '漢堡', '薑汁', '便當', '食物精緻漂亮', '其他正面情緒', '其他小吃']
狐狸尾巴(衛民店)
['綠蔓', '鮮奶', '阿花', '紫色', '珍珠', '飲料專賣店']
孤軍二代
['泰式', '蝦醬', '泰式料理', '食物美味', '泰國', '必必', '泰式料理']
O2 Brunch & Café 歐圖早午餐咖啡廚房(新市南科店)
['玄米', '勛', '市南', '科店', '其他正面情緒', '歐圖', '複合式']
蘇蝦仁肉圓
['建國', '碗粿', '蝦仁', '肉圓', '小馬', '麵食點心']
食光啡宿 Time Goes Fast
['正德', '老屋', '價格較高昂', '旅人', '食光啡宿', '迷及', '小份量', '咖啡專賣']
老夫子牛肉麵 成功店
['試用', '口味清淡', '環

['龍眼肉', '龍眼', '農庄', '龍眼乾', '特優', '休閒零食']
嘉祥粿仔
['咚滋咚', '跳針', '粿', '焦脆', '萬里', '中式早餐']
EZ kitchen 藝秝廚房
['南路', '便當', '入港', '食物美味', '柳枝', '秝', '價格較高昂', '咖啡專賣']
好物．私塾
['私塾', '老屋', '好物', '貓氏', '文具', '複合式']
登山廚司生活館
['登山', '嘉明湖', '避難', '向陽', '嚮導', '其他小吃']
淑聖廣東粥
['油條', '銘哥', '粥', '粥品', '廣東', '麵食點心']
沛里歐咖啡館
['帶酸好', '紅杯', '紫杯', '馬鈴薯', '香草', '咖啡專賣']
許記小館
['醃料', '五十', '伴隨', '咩', '著炸香', '價格較高昂', '台菜餐廳']
Ciao Bella Ristorante Italiano
['露燉', '煮飯', '最有', '唐明珍', '男人', '義式料理']
萬三米糕
['勁屋', '肉燥', '四神湯', '萬三', '米糕', '台菜餐廳']
旺角臭豆腐
['臭臭', '仙草', '臭豆腐', '小雞', '旺角', '其它中式料理']
OH!MISS.荳荳
['紫米', '果汁', '莎拉', '芭樂', '馬鈴薯', '果汁']
曉璘海產店
['牡蠣', '燒烤', '娃娃', '璘', '鳳螺', '食物美味', '海鮮餐廳']
雄さしみ沙西米專賣店
['沙西米', '檜木', '砧板', '頂真', '生魚片', '生魚片、壽司專賣']
喬美海鮮碳烤餐廳
['鰻魚', '喬美', '其他正面情緒', '月', '米糕', '月媽', '海鮮餐廳']
咖啡花朵
['優格', '勝利路', '土司', '咖啡茶', '蜜糖', '複合式']
囍思咖啡 SHE'S cafe'
['豐盛', '份量大', '囍', '謎思', '破費', '咖啡', '咖啡專賣']
六百番
['鮑魚', '海產', '魚翅', '水箱', '食物美味', '九孔', '其他亞洲料理']
冒煙的喬美式餐廳 台南店
['書局', '喬', '冒煙', '小莉', '二人', '美式料理']
營養薯大王
['糖醋', '薯條',

['阿輝', '鱔魚', '評個', '兩多', '炒', '熱炒、快炒']
小澤銅鑼燒
['燒店', '大福', '銅鑼', '蜜桃', '奶綴', '西點烘焙、麵包']
謬思園
['潛艇', '窯', '披薩', '鬆餅', '燻雞', '價格較高昂', '複合式']
壽賀喜屋(台南站前店)
['拉麵', '賀喜', '美食街', '蜜是', '叉燒', '其他日式料理']
精緻烘焙屋
['泡芙', '壯大', '別記', '豆穌', '三顧茅廬', '西點烘焙、麵包']
吳家米粿
['歸仁', '粿', '五十', '咩', '關廟', '其他小吃']
樸九鼎風味火鍋
['壽喜燒', '緹', '枸杞茶', '異味', '九鼎', '其他日式料理']
JT Bistor 西式餐酒館
['煮學', '烤當', '肋還', '升級', '食物美味', '這道', '其他異國料理']
新園A 汕頭火鍋
['東石', '美牛', '另一家', '新園', '小卷', '價格較高昂', '其他鍋類']
韓國先生(OBBA店)
['韓式料理', '價格較高昂', '先生', '韓國', '小毛', '泰瑞莎', '韓式料理']
日喜豆花
['冰版', '察看', '三十元', '粉條', '內用點', '剉冰、豆花']
麥午茶(麥午茶)
['小份量', '腸堡', '食物精緻漂亮', '鬆餅', '魚起司', '鮪', '麥午', '咖啡專賣']
花梅蘭壽司
['興華', '梅蘭', '蓋飯', '照燒', '壽司', '生魚片、壽司專賣']
85度C咖啡蛋糕烘焙專賣店(永康南科店)
['祝', '蛋糕', '生日快樂', '灰熊', '熊的頭', '價格較高昂', '複合式']
小公園水果店
['刈包', '鳳梨汁', '蜜餞', '圓環', '楊桃汁', '飲料專賣店']
郭記蒸好味湯包
['玩吧', '湯包', '贈獎', '巨鼠', '報報', '其它中式料理']
小新划島
['凍', '椰果', '小黃點', '喝人還', '綠綠的', '飲料專賣店']
左鎮電烤土雞
['連雞殼', '土雞', '簡易', '電烤', '畫功', '其他小吃']
麥當勞(台南東門店)
['小朋友', '經理', '課程', '麥當勞', '小麥', '漢堡、炸雞']
三媽臭臭鍋

['綠豆', '芋頭', '中杯', '薏仁', '其他正面情緒', '留言', '飲料專賣店']
卡路里 營養均衡早午餐(怡東店)
['她當', '卡路里', '其他正面情緒', '前後任', '食物不好吃', '第二任', '妹妹', '其他小吃']
A-Ru Cafe
['十字花科', '巴達', '為巧', '撤店', '瓷器', '複合式']
噠慷咖啡
['服務好', '宇治', '草莓', '千層', '抹茶', '食物美味', '鬆餅', '複合式']
三皇三家(文化店)
['蘇咪', '西莉亞', '撒哈', '魚或德式', '三皇', '價格較高昂', '複合式']
三峽牛角烘焙屋(台南崇善店)
['芺', '彎起來', '三峽', '有往', '牛角', '西點烘焙、麵包']
大上海臭臭鍋
['存在主義', '花園夜市', '盡而', '鍋品', '彼鄰', '其他鍋類']
欣福龍海鮮餐廳
['鄰近地區', '燴', '辦桌', '挑配', '福龍', '海鮮餐廳']
村家味素食豆簽羹
['簽羹', '剰', '湯鮮料', '村家味', '素豆', '小吃']
台南麻豆樂腳林草魚、虱目魚湯(粥)
['太遲', '魚湯', '划算', '肉燥飯', '過呀', '要厚著', '中式早餐']
DOUBLE囍南洋新式料理
['過味', '蔴', '相約', '趨使', '好奇心', '泰式料理']
陽光可樂餅
['勝利路', '炸蛋', '奶濃', '可樂', '無評', '蟹肉', '其他速食']
雞屎山學院
['老房子', '真空管', '環境比較不好', '雞屎', '其他正面情緒', '西瓜', '音響', '酒吧 Lounge Bar']
大水缸傳統飲品專賣店
['桂花', '酸梅湯', '想宅配', '號外', '中興新村', '剉冰、豆花']
學甲鵝肉
['網友', '杰在民雄', '竹筍湯', '不輸民雄', '鵝肉', '價格較高昂', '台菜餐廳']
模特兒陽春麵
['店究', '水餃', '裡又多', '前稱', '日新國', '麵食點心']
COCO 元元美式炸雞
['炸雞', '雞排', '元元', '胖拔', '食物很香', '排店', '漢堡、炸雞']
AWESOME二聖美式餐廳
['楔型', '薯條', '美式', '二

食味軒平價火鍋牛排館
['', '無評', '美式料理']
美國夏威夷毛伊咖啡館(小東店)
['', '無評', '咖啡專賣']
6218(麻豆店)
['', '無評', '咖啡專賣']
二氧化錳平價咖啡簡餐
['', '無評', '咖啡專賣']
知己簡餐
['', '無評', '咖啡專賣']
魔豆義式咖啡吧(善化店)
['', '無評', '咖啡專賣']
魔豆咖啡
['', '無評', '咖啡專賣']
閒靜咖啡館
['', '無評', '咖啡專賣']
飲淡薄原味海產
['', '無評', '海鮮餐廳']
沙鍋魚頭
['魚頭', '砂鍋', '份量大', '價格較高昂', '臉盆', '東山', '小份', '其他鍋類']
三媽臭臭鍋(永康永大店)
['', '無評', '其他鍋類']
良賓食堂
['', '無評', '台菜餐廳']
李老師牛肉麵
['', '無評', '麵食點心']
卓仔麵店
['', '無評', '麵食點心']
麵店
['拉麵', '滷味', '麵', '牛肉麵', '食物美味', '叉燒', '麵食點心']
東園手工水餃
['', '無評', '麵食點心']
乾爹牛肉麵
['', '無評', '麵食點心']
謝家羊肉
['', '無評', '麵食點心']
這一家北方麵食小館(新營店)
['', '無評', '麵食點心']
鍋貼
['煎餃', '餡餅', '酸辣湯', '葉子', '南園', '食物美味', '服務較差', '麵食點心']
饅頭世家
['', '無評', '麵食點心']
劉家麵食館
['', '無評', '麵食點心']
東北餃子館
['', '無評', '麵食點心']
家鄉鱔魚魯麵飯
['', '無評', '麵食點心']
DAVIDYANG coffee
['', '無評', '咖啡專賣']
新時代麵店
['', '無評', '麵食點心']
奇美餃子館
['', '無評', '麵食點心']
麵.飯
['', '無評', '麵食點心']
老楊手工水餃、麵、飯
['', '無評', '麵食點心']
賀好牛肉麵
['', '無評', '麵食點心']
張家麵館
['', '無評', '麵食點心']
禾富日本料理
['', '無評', '綜合日式料理']
好吉野日本料理
['', '無評', '綜合日式料理']
京都御所日式小火鍋店
[''

In [39]:
for i in BigAnalyzeTable:
    if i['score']>30 and i['Ncomment']>10:
        print(i['name']+' '+str(i['score']))
#         print(i['tags'])
        print(i['featurescores'])
        print('\n')
gooddien=[i for i in BigAnalyzeTable if i['score']>30 and i['Ncomment']>10]
# len(gooddien)

食下有約 想法廚房 34.61199999999995
{'食物美味': 1.36, '食物很香': 0.82, '環境好': 0.97, '食物精緻漂亮': 0.97, '份量大': 0.54, '其他正面情緒': 0.66, '划算': 0.18, '口味清淡': 0.07, '偏甜': 0.28, '服務好': 0.15, '重口味': 0.18, '小份量': 0.21, '偏辣': 0.06}


飛饗創意西式料理 36.16399999999993
{'食物美味': 1.33, '食物很香': 0.69, '划算': 0.31, '環境好': 1.06, '服務好': 0.51, '食物精緻漂亮': 1.28, '其他正面情緒': 0.85, '份量大': 0.65, '偏甜': 0.47, '口味清淡': 0.36, '偏酸': 0.8, '重口味': 0.31, '偏辣': 0.12, '小份量': 0.16}


小銅鍋義式餐廳(德安誠品店) 42.36956521739131
{'食物美味': 1.7, '食物很香': 0.99, '環境好': 1.13, '食物精緻漂亮': 0.71, '其他正面情緒': 0.62, '份量大': 0.65, '重口味': 0.45, '口味清淡': 0.24, '偏辣': 0.26, '划算': 0.18, '偏酸': 0.65, '服務好': 0.31, '偏甜': 0.11, '小份量': 0.17}


歐加里(台南大學店) 39.19999999999998
{'食物美味': 1.51, '食物很香': 0.72, '划算': 0.78, '環境好': 0.73, '食物精緻漂亮': 1.45, '其他正面情緒': 0.65, '份量大': 0.34, '小份量': 0.27, '重口味': 0.37, '偏辣': 0.12, '偏酸': 0.6, '口味清淡': 0.15, '服務好': 1.16, '偏甜': 0.33}


轉角餐廳 33.70909090909096
{'食物美味': 1.33, '食物很香': 1.0, '環境好': 1.01, '服務好': 0.6, '食物精緻漂亮': 0.82, '其他正面情緒': 0.65, '份量大': 0.33, '划算': -0.1, '重口味'

In [40]:
with open('./finish/bigtable_1.3.json','w') as f:
    json.dump(BigAnalyzeTable,f)
with open('D:/Data/JsonData/TainanFood/TainanGoodDien_1.1.json','w') as f:
    json.dump(gooddien,f)   

In [42]:
import json
with open('./finish/bigtable_1.3.json') as f:
    data=json.load(f)

In [44]:
data[5500]

{'Ncomment': 1,
 'TF_IDF': {'一大': 0.0034316562317740344,
  '一天': 0.010352436088502268,
  '一拿結': 0.020894584731729848,
  '一隻': 0.004654340183783867,
  '一點': 0.0015476692210728191,
  '三明治': 0.032810802257432055,
  '上學': 0.011225903123428492,
  '上班': 0.004861660621721897,
  '上班族': 0.006954570718779277,
  '不小': 0.004974051065529456,
  '以外': 0.004700852412062901,
  '但別': 0.014205029272530265,
  '來給': 0.008634440975997349,
  '來說': 0.0021777501661234423,
  '來頭': 0.00962023041125646,
  '個帳': 0.01824391109439839,
  '倍增': 0.01253264040773037,
  '先': 0.0012688791693471191,
  '全麥': 0.008920495634701726,
  '加入': 0.002467869442546391,
  '十分': 0.0032367995009867628,
  '半糖': 0.008489050785106771,
  '原汁': 0.0062252434152275035,
  '口味': 0.0023872542599381115,
  '台南市': 0.0008801262849506451,
  '吃': 0.004207440986911775,
  '各式各樣': 0.006065046506362557,
  '同事': 0.014380812803561599,
  '吐司': 0.013255888008187763,
  '喔': 0.0022386410681215366,
  '喝': 0.0011867766224660165,
  '囉': 0.0015564240445789148,
  '回謹

In [1]:
ls

 磁碟區 D 中的磁碟沒有標籤。
 磁碟區序號:  EABF-B91C

 D:\ForGitHub 的目錄

2017/09/16  下午 03:10    <DIR>          .
2017/09/16  下午 03:10    <DIR>          ..
2017/09/16  下午 03:06    <DIR>          .ipynb_checkpoints
2017/09/16  下午 03:08    <DIR>          data
2017/09/16  上午 09:51    <DIR>          dict
2017/09/16  下午 03:10    <DIR>          finish
2017/09/16  下午 03:10         1,185,812 newsummary.ipynb
2017/09/16  上午 10:36               264 README
2017/09/16  上午 09:53             5,821 redis.ipynb
               3 個檔案       1,191,897 位元組
               6 個目錄  476,277,907,456 位元組可用
